# Binary matrix factorization using D-Wave's quantum annealer

This notebook shows how to do binary matrix factorization (BMF) on D-Wave's 
quantum annealer. This is done by first formulating BMF as a quadratic 
unconstrained binary optimization (QUBO) problem, which is a type of problem 
that D-Wave's quantum annealer is designed to solve. The reformulation of BMF
into a QUBO follows the techniques presented by 
[Malik et al. (2021)](#references).

## Definition of binary matrix factorization

Consider a binary matrix $\bm A \in \{0,1\}^{m \times n}$. A rank-$r$ BMF of 
$\bm A$ takes the form $\bm A = \bm U \bm V^\top$ where 
$\bm U \in \{0,1\}^{m \times r}$ and $\bm V \in \{0,1\}^{n \times r}$, and where 
the $\bm U$ and $\bm V^\top$ are multiplied using standard matrix 
multiplication. Computing the rank-$r$ factorization of $\bm A$ can be 
formulated as the optimization problem

$$\min_{\bm U, \bm V} \| \bm A - \bm U \bm V^\top \|_\text{F}^2 \qquad 
\text{s.t.} \quad \bm U \in \{0,1\}^{m \times r}, 
\; \bm V \in \{0,1\}^{n \times r}.$$

As shown by [Malik et al. (2021)](#references),
this optimization problem can be reformulated into a QUBO which takes the form

$$\min_{\bm x} \bm x^\top \bm Q \bm x \qquad 
\text{s.t.} \quad \bm x \in \{0,1\}^k,$$

where $\bm Q \in \mathbb{R}^{k \times k}$ and $k$ depends on the variables 
$m$, $n$, and $r$. Once a solution vector $\bm x$ to the QUBO is computed, the
corresponding solution matrices $\bm U$ and $\bm V$ can be extracted from it.
[Malik et al. (2021)](#references) present two
different ways to formulate the BMF problem as QUBO problems; see the paper
for further details.

## Solving a small BMF problem using the quantum annealer

In this section, we run some tests on the small-scale BMF problem which appears 
in Example 1 by 
[Malik et al. (2021)](#references). Consider 
the matrix 

$$\bm A = 
\begin{bmatrix} 
    1 & 1 & 0 \\ 
    1 & 1 & 1 \\ 
    0 & 0 & 1 
\end{bmatrix}.$$

A rank-2 BMF of this matrix is given by $\bm A = \bm U \bm V^\top$ where

$$\bm U = 
\begin{bmatrix}
    0 & 1 \\
    1 & 1 \\
    1 & 0
\end{bmatrix},
\qquad
\bm V = 
\begin{bmatrix}
    0 & 1 \\
    0 & 1 \\
    1 & 0
\end{bmatrix}.$$

As a small starting example, we will decompose $\bm A$.
To that end, first define this matrix as a numpy array.

In [11]:
import numpy as np

A = np.array([[1, 1, 0], [1, 1, 1], [0, 0, 1]])
m, n = A.shape

Next, we use the functionality implemented in `qBMF` to formulate this as a QUBO
problem. We will first solve the
problem using D-Wave's simulated annealing sampler, which needs the QUBO to be 
provided in a dictionary format. In the code below, we return the QUBO 
corresponding to two different formulations.

In [12]:
import qBMF

# Set target rank
r = 2

# Set value of penalty variable
lam = 2.1*r*np.linalg.norm(A, ord='fro')**2

# Compute QUBO in dictionary format
qubo_dict_1 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="dict", formulation=1)
qubo_dict_2 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="dict", formulation=2)

Next, we solve both QUBO problems using D-Wave's simulated annealing sampler.

In [13]:
from dwave.samplers import SimulatedAnnealingSampler

# Construct sampler
sampler = SimulatedAnnealingSampler()

# Solve both problems using 1000 different samples each
sampleset_1 = sampler.sample_qubo(qubo_dict_1, num_reads=1000) 
sampleset_2 = sampler.sample_qubo(qubo_dict_2, num_reads=1000)

# Extract solution matrices and compute decomposition error
U_1, V_1 = qBMF.extract_U_V(sampleset_1, m, n, r)
U_2, V_2 = qBMF.extract_U_V(sampleset_2, m, n, r)
er_1 = np.linalg.norm(A - U_1 @ np.transpose(V_1))**2
er_2 = np.linalg.norm(A - U_2 @ np.transpose(V_2))**2

# Print results
print("Squared Frobenius Norm of A: {}".format(np.linalg.norm(A)**2))
print("Best Energy 1st Formulation: {}".format(sampleset_1.first.energy))
print("Best Energy 2nd Formulation: {}".format(sampleset_2.first.energy))
print("Factorization Error 1st Formulation: {}".format(er_1))
print("Factorization Error 2nd Formulation: {}".format(er_2))

Squared Frobenius Norm of A: 5.999999999999999
Best Energy 1st Formulation: -5.999999999999773
Best Energy 2nd Formulation: -4.999999999999204
Factorization Error 1st Formulation: 0.0
Factorization Error 2nd Formulation: 1.0


Since the solution generated by the annealing samplers are randomized, they may
not produce the correct output. However, the solution computed above is 
typically correct when `num_reads=1000` samples are used.

As noted in [Malik et al. (2021)](#references),
when `lam` (called $\lambda$ in the paper) is sufficiently large, if the 
solution $\bm x$ satisfies $\bm x^\top \bm Q \bm x = -\|\bm A\|_\text{F}^2$, 
then we know that the optimal solution has been found. As we can see above (in 
most cases when the code is run), the best energy is equal to the negative of 
the square of the norm of $\bm A$. Indeed, we see that this corresponds to zero
error in the factorization.

Next we will solve the problem above, but using D-Wave's quantum annealer 
instead. First, we compute the BQM representation of the BMF QUBO.

In [14]:
# Compute QUBO in dimod.BQM format
qubo_bqm_1 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="bqm", formulation=1)
qubo_bqm_2 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="bqm", formulation=2)

Next, we set up the quantum annealing solver.

In [15]:
from dwave.system import DWaveSampler, EmbeddingComposite

# Prepare quantum sampler
sampler = DWaveSampler()
embedding_sampler = EmbeddingComposite(sampler)
print("QPU {} was selected".format(sampler.solver.name))

QPU Advantage_system4.1 was selected


Finally, we solve the two QUBO formulations for BMF using D-Wave's sampler.  

In [16]:
sampleset_1 = embedding_sampler.sample(qubo_bqm_1, num_reads=1000, \
    label="QUBO BMF formul 1")
sampleset_2 = embedding_sampler.sample(qubo_bqm_2, num_reads=1000, \
    label="QUBO BMF formul 2")

# Extract solution matrices and compute decomposition error
U_1, V_1 = qBMF.extract_U_V(sampleset_1, m, n, r)
U_2, V_2 = qBMF.extract_U_V(sampleset_2, m, n, r)
er_1 = np.linalg.norm(A - U_1 @ np.transpose(V_1))**2
er_2 = np.linalg.norm(A - U_2 @ np.transpose(V_2))**2

# Print results
print("Squared Frobenius Norm of A: {}".format(np.linalg.norm(A)**2))
print("Best Energy 1st Formulation: {}".format(sampleset_1.first.energy))
print("Best Energy 2nd Formulation: {}".format(sampleset_2.first.energy))
print("Factorization Error 1st Formulation: {}".format(er_1))
print("Factorization Error 2nd Formulation: {}".format(er_2))

Squared Frobenius Norm of A: 5.999999999999999
Best Energy 1st Formulation: -4.999999999999972
Best Energy 2nd Formulation: -4.999999999999943
Factorization Error 1st Formulation: 1.0
Factorization Error 2nd Formulation: 1.0


As with the simulated annealing sampler, the quantum annealing sampler may not
always find the exact result. However, most of the time, the error should be 
zero.

## Attempting to solve a larger BMF problem

The problem in the previous section had the following number of binary 
parameters for each of the two QUBO formulations:

- Formulation 1: $k = (m + n + mn)r = 30$
- Formulation 2: $k = (m + n)(r + r^2) = 36$

We will now try solving slightly larger problems. A binary matrix $A$ of a 
specific size and rank can be generated using `qBMF.generate_binary_matrix` as
shown below.

In [17]:
# Size and binary rank of new matrix
m = 10
n = 8
r = 4

# Generate matrix
U, V = qBMF.generate_binary_matrix(m, n, r=r)
A = U @ np.transpose(V)
density = np.count_nonzero(A)/(m*n)
print("A is {}-by-{} and has a density of {}.".format(m, n, density))

# Size of QUBO problems
print("No. binary variables for formulation 1: {}".format((m + n + m*n)*r))
print("No. binary variables for formulation 2: {}".format((m + n)*(r + r**2)))


A is 10-by-8 and has a density of 0.375.
No. binary variables for formulation 1: 392
No. binary variables for formulation 2: 360


These problems have about 10 times as many binary variables to solve for as the 
smaller problem in the previous section. 
As discussed in 
[Malik et al. (2021)](#references), using a smaller value of `lam` ($\lambda$
in the paper) can be a good idea when solving harder problems. Although this 
means that the QUBO solution doesn't necessarily correspond to the best BMF,
it ends up
making the optimization problem more well-behaved and ultimately making it 
easier to find a good approximate solution. With this in mind, and following the
recommendation in [Malik et al. (2021)](#references), we set `lam=1` in the 
experiments below.

We first attempt to solve them using the simulated annealing sampler. 

In [18]:
# Set value of penalty variable
lam = 1

# Compute QUBO in dictionary format
qubo_dict_1 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="dict", formulation=1)
qubo_dict_2 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="dict", formulation=2)

# Construct sampler
sampler = SimulatedAnnealingSampler()

# Solve both problems using 1000 different samples each
sampleset_1 = sampler.sample_qubo(qubo_dict_1, num_reads=1000) 
sampleset_2 = sampler.sample_qubo(qubo_dict_2, num_reads=1000)

# Extract solution matrices and compute decomposition error
U_1, V_1 = qBMF.extract_U_V(sampleset_1, m, n, r)
U_2, V_2 = qBMF.extract_U_V(sampleset_2, m, n, r)
er_1 = np.linalg.norm(A - U_1 @ np.transpose(V_1))**2
er_2 = np.linalg.norm(A - U_2 @ np.transpose(V_2))**2

# Print results
print("Squared Frobenius Norm of A: {}".format(np.linalg.norm(A)**2))
print("Best Energy 1st Formulation: {}".format(sampleset_1.first.energy))
print("Best Energy 2nd Formulation: {}".format(sampleset_2.first.energy))
print("Factorization Error 1st Formulation: {}".format(er_1))
print("Factorization Error 2nd Formulation: {}".format(er_2))


Squared Frobenius Norm of A: 30.0
Best Energy 1st Formulation: -30.0
Best Energy 2nd Formulation: -134.0
Factorization Error 1st Formulation: 0.0
Factorization Error 2nd Formulation: 632.0


Next, we try solving the problem using the quantum annealer.

In [19]:
import dimod

# Compute QUBO in dimod.BQM format
qubo_bqm_1 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="bqm", formulation=1)
qubo_bqm_2 = qBMF.construct_BMF_BQM(A, r=r, lam=lam, format="bqm", formulation=2)

# Prepare quantum sampler
sampler = DWaveSampler()
embedding_sampler = EmbeddingComposite(sampler)
print("QPU {} was selected".format(sampler.solver.name))

sampleset_1 = embedding_sampler.sample(qubo_bqm_1, num_reads=1000, \
    label="QUBO BMF (Lrg) formul 1")
sampleset_2 = embedding_sampler.sample(qubo_bqm_2, num_reads=1000, \
    label="QUBO BMF (Lrg) formul 2")

# Extract solution matrices and compute decomposition error
U_1, V_1 = qBMF.extract_U_V(sampleset_1, m, n, r)
U_2, V_2 = qBMF.extract_U_V(sampleset_2, m, n, r)
er_1 = np.linalg.norm(A - U_1 @ np.transpose(V_1))**2
er_2 = np.linalg.norm(A - U_2 @ np.transpose(V_2))**2

# Print results
print("Squared Frobenius Norm of A: {}".format(np.linalg.norm(A)**2))
print("Best Energy 1st Formulation: {}".format(sampleset_1.first.energy))
print("Best Energy 2nd Formulation: {}".format(sampleset_2.first.energy))
print("Factorization Error 1st Formulation: {}".format(er_1))
print("Factorization Error 2nd Formulation: {}".format(er_2))


QPU Advantage_system4.1 was selected
Squared Frobenius Norm of A: 30.0
Best Energy 1st Formulation: -4.0
Best Energy 2nd Formulation: -96.0
Factorization Error 1st Formulation: 22.0
Factorization Error 2nd Formulation: 559.0


Finally, we also try using one of D-Wave's hybrid solvers to tackle the problem.

In [20]:
from dwave.system import LeapHybridSampler

sampler_hybrid = LeapHybridSampler()
sampleset_1 = sampler_hybrid.sample(qubo_bqm_1)
sampleset_2 = sampler_hybrid.sample(qubo_bqm_2)

# Extract solution matrices and compute decomposition error
U_1, V_1 = qBMF.extract_U_V(sampleset_1, m, n, r)
U_2, V_2 = qBMF.extract_U_V(sampleset_2, m, n, r)
er_1 = np.linalg.norm(A - U_1 @ np.transpose(V_1))**2
er_2 = np.linalg.norm(A - U_2 @ np.transpose(V_2))**2

# Print results
print("Squared Frobenius Norm of A: {}".format(np.linalg.norm(A)**2))
print("Best Energy 1st Formulation: {}".format(sampleset_1.first.energy))
print("Best Energy 2nd Formulation: {}".format(sampleset_2.first.energy))
print("Factorization Error 1st Formulation: {}".format(er_1))
print("Factorization Error 2nd Formulation: {}".format(er_2))

Squared Frobenius Norm of A: 30.0
Best Energy 1st Formulation: -30.0
Best Energy 2nd Formulation: -134.0
Factorization Error 1st Formulation: 0.0
Factorization Error 2nd Formulation: 527.0000000000001


Running the code above a few times, it seems like the simulated annealing and 
hybrid solvers frequently find an exact zero-error decomposition of $A$ when
the first formulation is used. The second formulations seems to yield worse 
results for all three solvers. The purely quantum based solver seems to perform
worse than the two other samplers. 

## References

O. A. Malik, H. Ushijima-Mwesigwa, A. Roy, A. Mandal, I. Ghosh. 
Binary matrix factorization on special purpose hardware. PLOS 
ONE 16(12): e0261250, 2021. DOI: 
[10.1371/journal.pone.0261250](https://doi.org/10.1371/journal.pone.0261250)